<a href="https://colab.research.google.com/github/AliaksandrSiarohin/first-order-model/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"

**Clone repository**

In [15]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256
Receiving objects: 100% (256/256), 72.13 MiB | 39.37 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [1]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### Separate Video and Audio (Optional)

```
!ffmpeg -y -i "/content/gdrive/My Drive/Work/DeepFake/unravel_template.mp4" -q:a 0 -map a "/content/gdrive/My Drive/Work/DeepFake/unravel_template_song.mp3"
```

## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

```
!ffmpeg -i /content/gdrive/My\ Drive/Work/DeepFake/bakamitai.mp4 -ss 00:08:57.50 -t 00:00:08 -filter:v "crop=600:600:760:50" -async 1 bakamitai2.mp4
```

In [2]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/Work/DeepFake/vox-cpk.pth.tar')

from demo import make_animation
from skimage import img_as_ubyte

In [3]:
reader = imageio.get_reader('/content/gdrive/My Drive/Work/DeepFake/unravel_template.mp4')
fps = reader.get_meta_data()['fps']

In [4]:
source_image = imageio.imread('/content/gdrive/My Drive/Work/DeepFake/my_cropped.png')
driving_video = imageio.mimread('/content/gdrive/My Drive/Work/DeepFake/unravel_template.mp4', memtest=False)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

/usr/local/lib/python3.6/dist-packages/imageio/core/format.py:411: UserWarning: Could not read last frame of /content/gdrive/My Drive/Work/DeepFake/unravel_template.mp4.
  warn("Could not read last frame of %s." % uri)
  0%|          | 0/1145 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/p

In [5]:
imageio.mimsave('/content/gdrive/My Drive/Work/DeepFake/generated_unravel_my.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

### Merge Audio and Video

In [6]:
!ffmpeg -y -i '/content/gdrive/My Drive/Work/DeepFake/generated_unravel_my.mp4' -i "/content/gdrive/My Drive/Work/DeepFake/unravel_template_song.mp3" -map 0 -map 1 -codec copy "/content/gdrive/My Drive/Work/DeepFake/generated_unravel_my_final.mp4"

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib